In [11]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [12]:
data = pd.read_csv('https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.EDUNF_STU_L2_TOT+EDUNF_STU_L2_TOT+EDUNF_STU_L3_TOT....?format=csv')

In [10]:
data

,Geographic area,Indicator,Sex,Age,Residence,Wealth Quintile,TIME_PERIOD,OBS_VALUE,Unit of measure,OBS_FOOTNOTE,Frequency,DATA_SOURCE,Unit multiplier,OBS_STATUS
0,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,1998,260160.0,Persons,NaN,Annual,UIS,Units,A
1,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,1999,261341.0,Persons,NaN,Annual,UIS,Units,A
2,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,2000,260718.0,Persons,NaN,Annual,UIS,Units,A
3,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,2001,266940.0,Persons,NaN,Annual,UIS,Units,A
4,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,2002,259650.0,Persons,NaN,Annual,UIS,Units,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2013,869167.0,Persons,NaN,Annual,UIS,Units,A
3337,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2014,834877.0,Persons,NaN,Annual,UIS,Units,A
3338,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2015,800425.0,Persons,NaN,Annual,UIS,Units,A
3339,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2016,768232.0,Persons,NaN,Annual,UIS,Units,A


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3341 entries, 0 to 3340
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geographic area  3341 non-null   object 
 1   Indicator        3341 non-null   object 
 2   Sex              3341 non-null   object 
 3   Age              3341 non-null   object 
 4   Residence        3341 non-null   object 
 5   Wealth Quintile  3341 non-null   object 
 6   TIME_PERIOD      3341 non-null   int64  
 7   OBS_VALUE        3341 non-null   float64
 8   Unit of measure  3341 non-null   object 
 9   OBS_FOOTNOTE     0 non-null      float64
 10  Frequency        3341 non-null   object 
 11  DATA_SOURCE      3341 non-null   object 
 12  Unit multiplier  3341 non-null   object 
 13  OBS_STATUS       3341 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 365.5+ KB


In [19]:
data.groupby(['Geographic area', 'TIME_PERIOD', 'Sex']).max()

Indicator  \
Geographic area TIME_PERIOD Sex                                                         
Albania         1998        Female  Lower Secondary education (ISCED 2) enrolments...   
                            Male    Lower Secondary education (ISCED 2) enrolments...   
                            Total   Upper secondary education (ISCED 3) enrolment ...   
                1999        Female  Upper secondary education (ISCED 3) enrolment ...   
                            Male    Upper secondary education (ISCED 3) enrolment ...   
...                                                                               ...   
Uzbekistan      2017        Male    Upper secondary education (ISCED 3) enrolment ...   
                            Total   Upper secondary education (ISCED 3) enrolment ...   
                2018        Female  Lower Secondary education (ISCED 2) enrolments...   
                            Male    Lower Secondary education (ISCED 2) enrolments...   
                            Total   Lower Secondary education (ISCED 2) enrolments...   

                                      Age Residence Wealth Quintile  \
Geographic area TIME_PERIOD Sex                                       
Albania         1998        Female  Total     Total           Total   
                            Male    Total     Total           Total   
                            Total   Total     Total           Total   
                1999        Female  Total     Total           Total   
                            Male    Total     Total           Total   
...                                   ...       ...             ...   
Uzbekistan      2017        Male    Total     Total           Total   
                            Total   Total     Total           Total   
                2018        Female  Total     Total           Total   
                            Male    Total     Total           Total   
                            Total   Total     Total           Total   

                                    OBS_VALUE Unit of measure  OBS_FOOTNOTE  \
Geographic area TIME_PERIOD Sex                                               
Albania         1998        Female   126246.0         Persons           NaN   
                            Male     133914.0         Persons           NaN   
                            Total    260160.0         Persons           NaN   
                1999        Female   127030.0         Persons           NaN   
                            Male     134311.0         Persons           NaN   
...                                       ...             ...           ...   
Uzbekistan      2017        Male    1253030.0         Persons           NaN   
                            Total   2433485.0         Persons           NaN   
                2018        Female  1210843.0         Persons           NaN   
                            Male    1286207.0         Persons           NaN   
                            Total   2497050.0         Persons           NaN   

                                   Frequency DATA_SOURCE Unit multiplier  \
Geographic area TIME_PERIOD Sex                                            
Albania         1998        Female    Annual         UIS           Units   
                            Male      Annual         UIS           Units   
                            Total     Annual         UIS           Units   
                1999        Female    Annual         UIS           Units   
                            Male      Annual         UIS           Units   
...                                      ...         ...             ...   
Uzbekistan      2017        Male      Annual         UIS           Units   
                            Total     Annual         UIS           Units   
                2018        Female    Annual         UIS           Units   
                            Male      Annual         UIS           Units   
                            Total     Annual         U

In [ ]:
fig = px.bar(data, x="TIME_PERIOD", y="OBS_VALUE", color="Indicator", template='plotly_dark')
fig.show()